<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/C%C3%A1lculo_do_Desconto_ou_Ganho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noise_kind.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_search_context.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_original_query.csv -P data

--2021-11-25 01:14:03--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12040 (12K) [text/plain]
Saving to: ‘util_bd_dataframe.py’

util_bd_dataframe.p 100%[===================>]  11.76K  --.-KB/s    in 0s      

2021-11-25 01:14:03 (55.0 MB/s) - ‘util_bd_dataframe.py’ saved [12040/12040]

--2021-11-25 01:14:04--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

# Obtenção da Base de dados

In [ ]:
import util_bd_dataframe  as util_bd_pandas

df_calculated_metric = util_bd_pandas.read_df_calculated_metric_with_label()[['search_context','cod_metric','noise_kind', 'value','cod_original_query']]
df_calculated_metric

,search_context,cod_metric,noise_kind,value,cod_original_query
0,TREC 2020 Judment-BM25,DCG@10,No changes in the text,1.746141,23849
1,TREC 2020 Judment-BM25,DCG@10,No changes in the text,7.000000,42255
2,TREC 2020 Judment-BM25,DCG@10,No changes in the text,24.412392,47210
3,TREC 2020 Judment-BM25,DCG@10,No changes in the text,7.315465,67316
4,TREC 2020 Judment-BM25,DCG@10,No changes in the text,3.014736,118440
...,...,...,...,...,...
4855,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.665893,1030303
4856,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.456374,1037496
4857,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.772823,1043135
4858,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.594450,1051399


# Dicionário guardando os valores de nDCG@10 para as queries originais, sem ruído

In [ ]:
nDCG10_original = df_calculated_metric.query('cod_metric == "nDCG@10" and noise_kind == "No changes in the text"').set_index(['cod_original_query', 'search_context'])[['value']].to_dict()['value']
list(nDCG10_original.items())[:10]

[((23849, 'TREC 2020 Judment-BM25'), 0.05490162180062975),
 ((42255, 'TREC 2020 Judment-BM25'), 0.37291464931616275),
 ((47210, 'TREC 2020 Judment-BM25'), 0.8662197873089202),
 ((67316, 'TREC 2020 Judment-BM25'), 0.27246345699710156),
 ((118440, 'TREC 2020 Judment-BM25'), 0.09478836436955079),
 ((121171, 'TREC 2020 Judment-BM25'), 0.07336392209936006),
 ((135802, 'TREC 2020 Judment-BM25'), 0.8630084885045989),
 ((141630, 'TREC 2020 Judment-BM25'), 0.3310440361812822),
 ((156498, 'TREC 2020 Judment-BM25'), 0.3408937879601314),
 ((169208, 'TREC 2020 Judment-BM25'), 0.29495549945770944)]

# Função para calcular o Desconto ou Ganho de uma query em um contexto de busca

In [ ]:
def calcular_desconto_ganho(cod_original_query, search_context, value):
    base = nDCG10_original[(cod_original_query, search_context)]
    if base:
        return value / base - 1
    return float("NaN")

# teste    
calcular_desconto_ganho(1030303, 'TREC 2020 Full-ReRank@1000', 0.676534)

-0.0456676209736564

# Cálculo de todos os valores de Desconto ou Ganho e armazenamento em um dataframe

In [ ]:
df_dg_nDCG10 = df_calculated_metric.query('cod_metric == "nDCG@10"')[['search_context', 'noise_kind', 'cod_original_query']]
df_dg_nDCG10['cod_metric'] = 'DG:nDCG@10'
df_dg_nDCG10['value'] = df_calculated_metric.query('cod_metric == "nDCG@10"').apply(lambda row:calcular_desconto_ganho(row.cod_original_query, row.search_context, row.value), axis = 1)
df_dg_nDCG10.dropna(inplace=True)
df_dg_nDCG10

,search_context,noise_kind,cod_original_query,cod_metric,value
54,TREC 2020 Judment-BM25,No changes in the text,23849,DG:nDCG@10,0.000000
55,TREC 2020 Judment-BM25,No changes in the text,42255,DG:nDCG@10,0.000000
56,TREC 2020 Judment-BM25,No changes in the text,47210,DG:nDCG@10,0.000000
57,TREC 2020 Judment-BM25,No changes in the text,67316,DG:nDCG@10,0.000000
58,TREC 2020 Judment-BM25,No changes in the text,118440,DG:nDCG@10,0.000000
...,...,...,...,...,...
4855,TREC 2020 Judment-ReRank@100,With one typographical error,1030303,DG:nDCG@10,0.023385
4856,TREC 2020 Judment-ReRank@100,With one typographical error,1037496,DG:nDCG@10,-0.506072
4857,TREC 2020 Judment-ReRank@100,With one typographical error,1043135,DG:nDCG@10,-0.001899
4858,TREC 2020 Judment-ReRank@100,With one typographical error,1051399,DG:nDCG@10,-0.279501


# TODO: guardar na base de dados

In [ ]:
### util_bd_pandas.save_calculated_metric(dict_val:dict, cod_search_context:int, cod_noise_kind:int)